<a href="https://colab.research.google.com/github/hermannvargens/SantanderBootcamp2023/blob/main/Explorando_IA_Generativa_em_um_Pipeline_de_ETL_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este primeiro desafio, a fim de seguir os três processos de Extração, Transformação e Carregamento (ETL), decidi criar um programa para o setor financeiro que irá analisar o valor da fatura de cartão de crédito de cada cliente e seu saldo bancário, em seguida ele verificará se há saldo suficente para pagamento da fatura e, caso não haja, deverá criar uma mensagem oferencendo empréstimo ao cliente.  
  
Uma vez que não encontrei um conjunto de dados com esses dados, especificamente, criei eu mesmo, utilizando a biblioteca "random" do Python. Em seguida guardei o dataset como um arquivo CSV, para posteriormente proceder a ETL.

# Criação do dataset

In [ ]:
import pandas as pd
import random

In [ ]:
ID = [ID for ID in range(11)]
print(ID)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
SaldoConta = [round(1000*random.random(),2) for saldo in ID]
print(SaldoConta)

[39.3, 963.75, 389.44, 559.16, 81.2, 15.67, 987.28, 64.19, 716.16, 454.85, 176.51]


In [ ]:
FaturaCartao = [round(1000*random.random(),2) for fatura in ID]
print(FaturaCartao)

[740.36, 128.6, 758.26, 637.64, 683.03, 962.01, 354.49, 468.37, 286.11, 163.26, 265.17]


In [ ]:
dados_clientes = pd.DataFrame({'ID':ID, 'SaldoConta':SaldoConta,'FaturaCartao':FaturaCartao})
print(dados_clientes)

    ID  SaldoConta  FaturaCartao
0    0       39.30        740.36
1    1      963.75        128.60
2    2      389.44        758.26
3    3      559.16        637.64
4    4       81.20        683.03
5    5       15.67        962.01
6    6      987.28        354.49
7    7       64.19        468.37
8    8      716.16        286.11
9    9      454.85        163.26
10  10      176.51        265.17


In [ ]:
try:
  dados_clientes.to_csv('dados_clientes.csv', index=False)
  print('Dados gravados em arquivo csv.')
except:
  print('Os dados não foram gravados.')

Dados gravados em arquivo csv.


# Extract

Vamos extrair os dados utilizando o método "read_csv" da biblioteca Pandas:

In [ ]:
dados_clientes = pd.read_csv('dados_clientes.csv')
dados_clientes

,ID,SaldoConta,FaturaCartao
0,0,39.30,740.36
1,1,963.75,128.60
2,2,389.44,758.26
3,3,559.16,637.64
4,4,81.20,683.03
5,5,15.67,962.01
6,6,987.28,354.49
7,7,64.19,468.37
8,8,716.16,286.11
9,9,454.85,163.26


 # Transform

Vamos agora realizar uma transformação nos dados, utilizando o Google Bard, outro grande modelo de linguagem, semelhante ao ChatGPT da OpenAI, para gerar mensagens de marketing para os clientes do banco que necessitam de empréstimo.  
  
A função irá definir se o cliente precisa ou não de empréstimo para os casos em que o Saldo da Conta ('SaldoConta') for menor que o valor da Fatura do Cartão ('FaturaCartão').

In [ ]:
from bardapi import Bard

token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' #apagado por motivos de segurança
bard = Bard(token=token)

In [ ]:
def mensagem(ID):
  return "Cliente ok!" if dados_clientes['SaldoConta'][ID] > dados_clientes['FaturaCartao'][ID] else bard.get_answer("Crie uma mensagem de marketing de apenas 1 linha oferecendo um empréstimo para pagar as contas.")['content']

In [ ]:
#Agora utilizamos a função MAP para aplicar a função criada (status) para cada ID

dados_clientes['Status'] = list(map(mensagem,dados_clientes['ID']))
print(dados_clientes)

    ID  SaldoConta  FaturaCartao  \
0    0       39.30        740.36   
1    1      963.75        128.60   
2    2      389.44        758.26   
3    3      559.16        637.64   
4    4       81.20        683.03   
5    5       15.67        962.01   
6    6      987.28        354.49   
7    7       64.19        468.37   
8    8      716.16        286.11   
9    9      454.85        163.26   
10  10      176.51        265.17   

                                               Status  
0   Aqui está uma mensagem de marketing de 1 linha...  
1                                         Cliente ok!  
2   Aqui está uma mensagem de marketing de 1 linha...  
3   Aqui está uma mensagem de marketing de 1 linha...  
4   **Pague suas contas em dias.**\n\nEssa mensage...  
5   **Dinheiro para as contas.**\n\nEssa mensagem ...  
6                                         Cliente ok!  
7   **Contas em dia, vida em dia.**\n\nEssa mensag...  
8                                         Cliente ok!  
9      

Vamos observar um exemplo do texto inteiro obtido com o Google Bard.  
Verificamos que embora tenhamos pedido que ele criasse uma mensagem com apenas 1 linha, foram retornados várrias outras mensagens e explicações.

In [ ]:
dados_clientes['Status'][0]

'Aqui está uma mensagem de marketing de 1 linha oferecendo um empréstimo para pagar as contas:\n\n**Contas em atraso? Peça um empréstimo hoje!**\n\nEssa mensagem é simples e direta, e vai direto ao ponto. Ela chama a atenção do leitor e transmite a mensagem de que o empréstimo é uma solução para quem está com contas em atraso.\n\nAqui está outra opção:\n\n**Empréstimo rápido para pagar as contas**\n\nEssa mensagem também é simples e direta, mas enfatiza a velocidade do processo de aprovação do empréstimo. Isso pode ser atraente para quem precisa de dinheiro com urgência.\n\nEm última análise, a melhor mensagem de marketing dependerá do público-alvo e dos objetivos da empresa. No entanto, essas duas opções são um bom ponto de partida.'

In [ ]:
dados_clientes['Status'][2]

'Aqui está uma mensagem de marketing de 1 linha oferecendo um empréstimo para pagar as contas:\n\n**Pague as contas com um empréstimo rápido e fácil.**\n\nEssa mensagem é simples e direta, e vai direto ao ponto. Ela chama a atenção do leitor e transmite a mensagem de que o empréstimo é uma solução rápida e fácil para pagar as contas.\n\nOutra opção é:\n\n**Empréstimo para pagar as contas**\n\nEssa mensagem é ainda mais curta e direta, mas ainda transmite a mensagem de que o empréstimo é uma solução para quem precisa de dinheiro para pagar as contas.\n\nA melhor mensagem de marketing dependerá do público-alvo e dos objetivos da empresa. No entanto, essas duas opções são um bom ponto de partida.'

Verificamos que há um padrão nas respostas, e o texto que nos interessa se encontra entre '**'.  
Vamos então criar uma função para extrai-lo, usando o método find:

In [ ]:
def extrair_texto(texto):

  inicio = texto.find('**') # para encontrar a sequência de caracteres '**'
  fim = texto.find('**', inicio + 2) # para encontrar a próxima sequência de caracteres '**'

  if inicio == -1 or fim == -1: # caso a função não encontre '**':
    return 'O banco agradece por ter você como nosso cliente!'
  else:
    return texto[inicio + 2:fim] # retorna o texto entre os caracteres '**'

Vamos utilizar a função map novamente para criar uma coluna com as mensagens que extrairmos:

In [ ]:
dados_clientes['Mensagem'] = list(map(extrair_texto,dados_clientes['Status']))
dados_clientes

,ID,SaldoConta,FaturaCartao,Status,Mensagem
0,0,39.30,740.36,Aqui está uma mensagem de marketing de 1 linha...,Contas em atraso? Peça um empréstimo hoje!
1,1,963.75,128.60,Cliente ok!,O banco agradece por ter você como nosso cliente!
2,2,389.44,758.26,Aqui está uma mensagem de marketing de 1 linha...,Pague as contas com um empréstimo rápido e fácil.
3,3,559.16,637.64,Aqui está uma mensagem de marketing de 1 linha...,Empréstimo para emergências.
4,4,81.20,683.03,**Pague suas contas em dias.**\n\nEssa mensage...,Pague suas contas em dias.
5,5,15.67,962.01,**Dinheiro para as contas.**\n\nEssa mensagem ...,Dinheiro para as contas.
6,6,987.28,354.49,Cliente ok!,O banco agradece por ter você como nosso cliente!
7,7,64.19,468.37,"**Contas em dia, vida em dia.**\n\nEssa mensag...","Contas em dia, vida em dia."
8,8,716.16,286.11,Cliente ok!,O banco agradece por ter você como nosso cliente!
9,9,454.85,163.26,Cliente ok!,O banco agradece por ter você como nosso cliente!


Agora apagaremos a coluna 'Status':

In [ ]:
dados_clientes.drop(columns='Status', inplace = True)
dados_clientes

,ID,SaldoConta,FaturaCartao,Mensagem
0,0,39.30,740.36,Contas em atraso? Peça um empréstimo hoje!
1,1,963.75,128.60,O banco agradece por ter você como nosso cliente!
2,2,389.44,758.26,Pague as contas com um empréstimo rápido e fácil.
3,3,559.16,637.64,Empréstimo para emergências.
4,4,81.20,683.03,Pague suas contas em dias.
5,5,15.67,962.01,Dinheiro para as contas.
6,6,987.28,354.49,O banco agradece por ter você como nosso cliente!
7,7,64.19,468.37,"Contas em dia, vida em dia."
8,8,716.16,286.11,O banco agradece por ter você como nosso cliente!
9,9,454.85,163.26,O banco agradece por ter você como nosso cliente!


# LOAD

Finalmente, podemos carregar os novos dados para um arquivo CSV:

In [ ]:
try:
  dados_clientes.to_csv('dados_clientes_atualizados.csv', index=False)
  print('Dados gravados em arquivo csv.')
except:
  print('Os dados não foram gravados.')

Dados gravados em arquivo csv.
